In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from numpy import asarray
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [2]:
# Laddar upp datasetet. sep är för att tala om att det är delat med ;
dataSet = pd.read_csv('DataCSV.csv', sep=';', header =1)

In [3]:
# Tar bort alla deciamlkomma som störde ut hela filen
colsToDrop = [col for col in dataSet.columns if 'Unnamed' in col]
dataSet = dataSet.drop(columns=colsToDrop)

In [4]:
# Skriver ut allt å ser så det är bra.
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print (dataSet)

      SFH  popUpWidnow  SSLfinal_State  Request_URL  URL_of_Anchor  \
0       1           -1               1           -1             -1   
1      -1           -1              -1           -1             -1   
2       1           -1               0            0             -1   
3       1            0               1           -1             -1   
4      -1           -1               1           -1              0   
5      -1           -1               1           -1             -1   
6       1           -1               0            1             -1   
7       1            0               1            1              0   
8      -1           -1               0           -1             -1   
9      -1            0              -1           -1              1   
10     -1           -1               0           -1             -1   
11      1            0               1            1              1   
12      1           -1               0           -1              1   
13      1           

In [5]:
# Konverterar datan till float för att kunna normalisera
columnsScale = dataSet.columns
dataSet[columnsScale] = dataSet[columnsScale].astype('float64')

In [ ]:
# Normaliserar datan med MinMaxScaler. [0,1]
scaler = MinMaxScaler()
dsNormalizedArray = scaler.fit_transform(dataSet[columnsScale])
dataSetNormalized = pd.DataFrame(dsNormalizedArray, columns=columnsScale)

In [ ]:
# Skriver ut filen normaliserad. 
print(dataSetNormalized)

      SFH  popUpWidnow  SSLfinal_State  Request_URL  URL_of_Anchor  \
0     1.0          0.0             1.0          0.0            0.0   
1     0.0          0.0             0.0          0.0            0.0   
2     1.0          0.0             0.5          0.5            0.0   
3     1.0          0.5             1.0          0.0            0.0   
4     0.0          0.0             1.0          0.0            0.5   
5     0.0          0.0             1.0          0.0            0.0   
6     1.0          0.0             0.5          1.0            0.0   
7     1.0          0.5             1.0          1.0            0.5   
8     0.0          0.0             0.5          0.0            0.0   
9     0.0          0.5             0.0          0.0            1.0   
10    0.0          0.0             0.5          0.0            0.0   
11    1.0          0.5             1.0          1.0            1.0   
12    1.0          0.0             0.5          0.0            1.0   
13    1.0          0

In [9]:
# Dela upp datan i target(y) och alla futeres(x) som den ska tränas på. 

# *** Orginaldata
targetColumn = 'Resultat'
y = dataSet[targetColumn]
x = dataSet.drop(targetColumn, axis=1)

# *** Normaliserad data
targetColumns = 'Resultat'
yN = dataSetNormalized[targetColumns]
xN = dataSetNormalized.drop(targetColumns, axis=1)


In [ ]:
# Skriver ut de olka dataseten
print("Orginaldata:                      dataSet")
print("Normaliserad data:                dataSetNormalized")
print()
print("Target data Orginal:              y")
print("FuturesData Orginal:              x")
print()
print("Target data Normaliserad[0,1]:    yN")
print("FuturesData Normaliserad[0,1]:    xN")

Orginaldata:                      dataSet
Normaliserad data:                dataSetNormalized

Target data Orginal:              y
FuturesData Orginal:              x

Target data Normaliserad[0,1]:    yN
FuturesData Normaliserad[0,1]:    xN


In [18]:
# Nu är det dags att dela upp i träningsdata och testdata

# Vi börjar med 90/10 enligt uppgift.
xTrainData, xTestData, yTrainData, yTestData  = train_test_split(x, y, test_size= 0.1, random_state=None)


In [ ]:
# Vi skriver ut hur fördelningen blev med 90/10
print("xTrainData shape: {}\nyTrainData shape: {}".format(xTrainData.shape, yTrainData.shape))
print("xTestData shape: {}\nyTestData shape: {}".format(xTestData.shape, yTestData.shape))

xTrainData shape: (1217, 9)
yTrainData shape: (1217,)
xTestData shape: (136, 9)
yTestData shape: (136,)


In [ ]:
# Vi delar upp datan i 70/30
xTrainData, xTestData, yTrainData, yTestData  = train_test_split(x, y, test_size= 0.3, random_state=None)


In [ ]:
# Vi skriver ut hur fördelningen blev med 70/30
print("xTrainData shape: {}\nyTrainData shape: {}".format(xTrainData.shape, yTrainData.shape))
print("xTestData shape: {}\nyTestData shape: {}".format(xTestData.shape, yTestData.shape))

xTrainData shape: (947, 9)
yTrainData shape: (947,)
xTestData shape: (406, 9)
yTestData shape: (406,)
